![Degirum banner](https://raw.githubusercontent.com/DeGirum/PySDKExamples/main/images/degirum_banner.png)
## Managing a Parking Lot in a Video Frame-by-Frame
This notebook demonstrates the management of a parking lot in a video.

In each video frame, user-defined polygon zones are checked for occupancy. Vehicles are detected in each frame, and the counts of occupied and available zones are noted. This information is used to annotate a video.

This script works with the following inference options:

1. Run inference on the DeGirum Cloud Platform;
2. Run inference on a DeGirum AI Server deployed on the local host or on some computer in your LAN or VPN;
3. Run inference on a DeGirum ORCA accelerator directly installed on your computer.

To try different options, you need to specify the appropriate `hw_location` option. 

When running this notebook locally, you need to specify your cloud API access token in the [env.ini](../../env.ini) file, located in the same directory as this notebook.

When running this notebook in Google Colab, the cloud API access token should be stored in a user secret named `DEGIRUM_CLOUD_TOKEN`.

Note: Please specify a path to an input video source before running this notebook.

In [ ]:
# make sure degirum and degirum-tools packages are installed
!pip show degirum-tools || pip install degirum-tools

**NOTE: this notebook requires a zones JSON file. It can be generated with the help of a GUI annotation tool provided by DeGirum. Please refer to the 'zone_annotation.ipynb' notebook for a demonstration of this utility.**

#### Specify where you want to run your inferences, model zoo url, model name, path to zones JSON file, and video source

NOTE: The `zones_json_name` argument MUST be specified by the user. All of the other arguments may remain as defined below.

In [ ]:

# hw_location: where you want to run inference
#     "@cloud" to use DeGirum cloud
#     "@local" to run on local machine
#     IP address for AI server inference
# vehicle_model_zoo_url: url/path for vehicle detection model zoo
#     Use cloud_zoo_url for @cloud, @local, and AI server inference options.
#     Use '' for an AI server serving models from a local folder.
#     Use a path to a JSON file for a single model zoo in case of @local inference.
# vehicle_model_name: name of the model for person detection.
# zones_json_name: path to zone JSON file
# video_source: video source for inference
#     camera index for local camera
#     URL of RTSP stream
#     URL of YouTube Video
#     path to video file (mp4 etc)
# output_video: annotated video destination
# degirum_cloud_token: your token for accessing the DeGirum cloud platform
hw_location = "@cloud"
vehicle_model_zoo_url = "https://cs.degirum.com/degirum/visdrone"
vehicle_model_name = "yolov8s_relu6_visdrone--640x640_float_openvino_cpu_1"
zones_json_name = "<path to JSON file>"             # <--- PLEASE DEFINE
video_source = "https://raw.githubusercontent.com/DeGirum/PySDKExamples/main/images/Parking.mp4"
output_video = "temp/Parking_annotated.mp4"

#### Specify arguments for the annotation process
Specify the occupied and vacant zone colors below, as well as the objects to be considered for detection.

The ZoneCounter algorithm supports the following zone triggering conditions:
 1. User-defined point(s) on the object's bounding box is/are found inside the zone.
 2. The object bounding box's intersection over polygon area (IoPA) is greater than a user-defined threshold.

To use the first approach, set the `use_triggers` flag below to True, and assign the list of triggering position(s) to `triggering_position_values`.
To use the second approach, set the `use_triggers` flag to False, and specify `iopa_threshold`.

The algorithm can optionally resize the detected object bounding boxes, scaling the width and height
around the center point of the bounding box, before triggering the zones.
Specify `bounding_box_scale` to use this feature.

In order to reduce fluctuations in the output of the algorithm, due to the event when an object is initially appears, disappears for a short period of time
in a video stream, and then reappears, information from the ObjectTracker algorithm can be used in the ZoneCounter if the `use_tracking` flag is set to True. For each detected object, the bounding box coordinates of that object in a user-defined number of prior frames is saved by the ObjectTracker algorithm, and the ZoneCounter can use this information to trigger a zone with an object that may not be necessarily present in the current frame, but was present a short while ago. The `track_buffer` variable can be assigned with the depth (in number of frames) of an object's position history.

The output video can be optionally annotated with the detection model's results, if the `show_ai_overlay` flag is set to True.

In [ ]:
import degirum_tools

occupied_zone_color = (0, 0, 255)
available_zone_color = (0, 255, 0)
class_list = ["car", "van", "truck", "bus"]

use_triggers = True
# if use_triggers is True, the trigger-point approach is used
triggering_position_values = [degirum_tools.AnchorPoint.CENTER]
triggering_positions = triggering_position_values if use_triggers else None
# else, the IoPA approach is used
iopa_threshold = 0.5

bounding_box_scale = 1.0

use_tracking = True
track_buffer = 30

show_ai_overlay = False

#### Load the model

In [ ]:
import degirum as dg

# Load and configure model
model = dg.load_model(
    vehicle_model_name,
    hw_location,
    vehicle_model_zoo_url,
    degirum_tools.get_token(),
    image_backend="opencv",
    overlay_show_labels=False,
    overlay_line_width=2,
    overlay_font_scale=2,
    output_confidence_threshold=0.4,
    overlay_show_probabilities=False,
    output_max_detections=300,
    output_max_detections_per_class=300,
    input_letterbox_fill_color=(114, 114, 114),
    output_class_set=set(class_list)
)

#### Analyzers
In order to annotated a video, a class, called an Analyzer, is required. A child of this class, the ZoneCounter, is implemented in DeGirum tools, and keeps track of the number and type of objects in pre-defined polygon zones. A child class of the ZoneCounter needs to be created to determine the occupied and available zones. This class, called ZoneOccupancyCounter, is implemented below.
Additionally, to allow the ZoneCounter to use object tracking information, the ObjectTracker is instantiated.

In [ ]:
import numpy as np
import json, cv2


class ZoneOccupancyCounter(degirum_tools.ZoneCounter):
    def __init__(
        self,
        polygon_json,
        triggering_positions,
        bounding_box_scale,
        iopa_threshold,
        occupied_zone_color,
        available_zone_color,
        class_list,
        use_tracking
    ):
        with open(polygon_json, "r") as poly_json:
            self.polygon_json = json.load(poly_json)
            self.polygon_json = [zone for zone in self.polygon_json["objects"]]
        self.occupied_zone_color = occupied_zone_color
        self.available_zone_color = available_zone_color
        super().__init__(
            count_polygons=self.polygon_json,
            class_list=class_list,
            triggering_position=triggering_positions,
            bounding_box_scale=bounding_box_scale,
            iopa_threshold=iopa_threshold,
            use_tracking=use_tracking,
            window_name="Vehicle Management"
        )

    def annotate(self, result, image: np.ndarray) -> np.ndarray:
        total_slots, filled_slots = len(self.polygon_json), 0
        empty_slots = total_slots

        # draw annotations
        for zi in range(len(self._polygons)):
            region_occupied = result.zone_counts[zi].get('total', 0) > 0
            line_color = self.occupied_zone_color if region_occupied else self.available_zone_color
            cv2.polylines(
                image, [self._polygons[zi]], True, line_color, result.overlay_line_width
            )
            if region_occupied:
                filled_slots += 1
                empty_slots -= 1

        label = "Occupancy: {}\nAvailable: {}".format(filled_slots, empty_slots)
        
        back_color = (255, 255, 255)
        font_color = degirum_tools.deduce_text_color(back_color)
        degirum_tools.put_text(
            image,
            label,
            (image.shape[1], 0),
            corner_position=degirum_tools.CornerPosition.TOP_RIGHT,
            bg_color=back_color,
            font_color=font_color,
            font_scale=result.overlay_font_scale,
            font_thickness=4,
            line_spacing=1.5
        )
        return image
    

# Instantiate a ZoneOccupancyCounter Analyzer
zone_occupancy_counter = ZoneOccupancyCounter(
    zones_json_name,
    triggering_positions,
    bounding_box_scale,
    iopa_threshold,
    occupied_zone_color,
    available_zone_color,
    class_list,
    use_tracking
)

# Instantiate an ObjectTracker Analyzer
tracker = degirum_tools.ObjectTracker(
    class_list=class_list,
    track_buffer=track_buffer,
    trail_depth=track_buffer,
    show_overlay=False
)

#### Annotate a video source with Analyzer(s).

In [ ]:
# annotate video
analyzers = [tracker, zone_occupancy_counter] if use_tracking else [zone_occupancy_counter]
degirum_tools.annotate_video(model, video_source, output_video, visual_display=True, show_ai_overlay=show_ai_overlay, analyzers=analyzers)